<a href="https://colab.research.google.com/github/Evilzero16/DL/blob/main/GD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import copy
from numpy.random import permutation

class Line():
    """
        Linear Model with two weights w0 (intercept) and w1 (slope)
    """
    def __init__(self):
        self.weights = [np.random.uniform(0,1,1) for _ in range(2)]
        self.derivative_funcs = [self.dx_w0, self.dx_w1]
        
    def evaluate(self,x):
        """
            evaluate: will evaluate the line yhate given x
            x: a point in the plane

            return the result of the function evalutation
        """
        return self.weights[0] + self.weights[1]*x

    def derivate(self, x, y):
        """
            derivate: will calculate all partial derivatives and return them
            input:
            x: a point in the plane
            y: the response of the point x
            
            output:
            partial_derivatives: an array of partial derivatives
        """
        partial_derivatives = []
        
        yhat = self.evaluate(x)
        partial_derivatives.append(self.dx_w0(x, y, yhat))
        partial_derivatives.append(self.dx_w1(x, y, yhat))
        
        return partial_derivatives
    
    def dx_w0(self, x, y, yhat):
        """
            dx_w0: partial derivative of the weight w0
            x: a point in the plane
            y: the response of the point x
            yhat: the current approximation of y given x and the weights

            return the gradient at that point for this x and y for w0
        """
        return 2*(yhat - y)
    
    def dx_w1(self, x, y, yhat):
        """
            dx_w1: partial derivative of the weight w1 for a linear function
            x: a point in the plane
            y: the response of the point x
            yhat: the current approximation of y given x and the weights

            return the gradient at that point for this x and y for w1
        """  
        return 2*x*(yhat - y)

    def __str__(self):
        return f"y = {self.weights[0]} + {self.weights[1]}*x"
        
    
#################### Helper functions ######################
def stochastic_sample(xs, ys):
    """
        stochastic_sample: sample with replacement one x and one y
        xs: all point on the plane
        ys: all response on the plane
        
        return the randomly selected x and y point
    """
    perm = permutation(len(xs))
    x = xs[perm[0]]
    y = ys[perm[0]]

    return x, y
    
    
def gradient(dx, evaluate, xs, ys):
    """
        gradient: estimate mean gradient over all point for w1
        evaluate: the evaulation function from the model
        dx: partial derivative function used to evaluate the gradient
        xs: all point on the plane
        ys: all response on the plane
        
        return the mean gradient all x and y for w1
    """         
    N = len(ys)
    
    total = 0
    for x,y in zip(xs,ys):
        yhat = evaluate(x)
        total = total + dx(x, y, yhat)
    
    gradient = total/N
    return gradient

################## Optimization Functions #####################

def gd(model, xs, ys, learning_rate = 0.01, max_num_iteration = 1000):
    """
        gd: will estimate the parameters w1 and w2 (here it uses least square cost function)
        model: the model we are trying to optimize using gradient descent
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
    """    

    for i in range(max_num_iteration):
        # Updating the model parameters
        model.weights = [weight - learning_rate*gradient(derivative_func, model.evaluate, xs, ys) for weight, derivative_func in zip(model.weights, model.derivative_funcs)]
        
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
def sgd(model, xs, ys, learning_rate = 0.01, max_num_iteration = 1000):
    """
        sgd: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
    """       
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        # Updating the model parameters
        model.weights = [weight - learning_rate*derivative for weight, derivative in zip(model.weights, model.derivate(x,y))]        
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
def sgd_momentum(model, xs, ys, learning_rate = 0.01, decay_factor = 0.9, max_num_iteration = 1000):
    """
        sgd_momentum: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        decay_factor: determines the relative contribution of the current gradient and earlier gradients to the weight change
        max_num_iteration: the number of iteration before we stop updating
    """
    
    # Create the gradient that we keep track as an array of 0 of the same size as the number of weights
    gradients = [0 for _ in range(len(model.weights))]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)

        # Calculate the new gradients
        gradients = [decay_factor*g + learning_rate*derivative for g, derivative in zip(gradients, model.derivate(x,y))]
        
        # Updating the model parameters
        model.weights = [weight - g for weight, g in zip(model.weights, gradients)]
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
            
def adagrad(model, xs, ys, learning_rate = 0.1, max_num_iteration = 1000, eps=0.0000001):
    """
        adagrad: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
        eps: is a numerical safety to avoid division by 0
    """         
    
    # Here only the diagonal matter
    num_param = len(model.weights)
    G = [[0 for _ in range(num_param)] for _ in range(num_param)]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        # Update G and the model weights iteratively (Note: speed up could be gained from vectorized implementation)
        for idx, gradient in enumerate(model.derivate(x, y)):
            G[idx][idx] = G[idx][idx] + gradient**2
            model.weights[idx] = model.weights[idx] - (learning_rate / np.sqrt(G[idx][idx] + eps)) * gradient
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
def rmsprop(model, xs, ys, learning_rate = 0.01, decay_factor = 0.9, max_num_iteration = 10000, eps=0.0000001):
    """
        rmsprop: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        decay_factor: the parameter used in the running averaging
        max_num_iteration: the number of iteration before we stop updating
        eps: is a numerical safety to avoid division by 0
    """         
    
    # Running average
    E = [0 for _ in range(len(model.weights))]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        # Update E and the model weights iteratively (Note: speed up could be gained from vectorized implementation)
        for idx, gradient in enumerate(model.derivate(x, y)):    
            E[idx] = decay_factor*E[idx] + (1 - decay_factor)*(gradient**2)
            model.weights[idx] = model.weights[idx] - (learning_rate/np.sqrt(E[idx] + eps))*gradient

    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)

def adam(model, xs, ys, learning_rate = 0.1, b1 = 0.9, b2 = 0.999, epsilon = 0.00000001, max_iteration = 1000):
    """
        Adam: This is the adam optimizer that build upon adadelta and RMSProp
        model: The model we want to optimize the parameter on
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the amount of learning we want to happen at each time step (default is 0.1 and will be updated by the optimizer)
        b1: this is the first decaying average with proposed default value of 0.9 (deep learning purposes)
        b2: this is the second decaying average with proposed default value of 0.999 (deep learning purposes)
        epsilon: a variable for numerical stability during the division
        max_iteration: the number of sgd round we want to do before stopping the optimization
    """
    
    
    # Variable Initialization
    num_param = len(model.weights)
    m = [0 for _ in range(num_param)] # two m for each parameter
    v = [0 for _ in range(num_param)] # two v for each parameter
    g = [0 for _ in range(num_param)] # two gradient
    
    for t in range(1,max_iteration):
        
        # Calculate the gradients 
        x, y = stochastic_sample(xs, ys)
        
        # Get the partial derivatives
        g = model.derivate(x, y)

        # Update the m and v parameter
        m = [b1*m_i + (1 - b1)*g_i for m_i, g_i in zip(m, g)]
        v = [b2*v_i + (1 - b2)*(g_i**2) for v_i, g_i in zip(v, g)]

        # Bias correction for m and v
        m_cor = [m_i / (1 - (b1**t)) for m_i in m]
        v_cor = [v_i / (1 - (b2**t)) for v_i in v]

        # Update the parameter
        model.weights = [weight - (learning_rate / (np.sqrt(v_cor_i) + epsilon))*m_cor_i for weight, v_cor_i, m_cor_i in zip(model.weights, v_cor, m_cor)]
        
        if t % 100 == 0:
            print(f"Iteration {t}")
            print(model)
    
def nesterov(model, xs, ys, learning_rate = 0.01, decay_factor = 0.9, max_num_iteration = 1000):
    """
        Nesterov: This is the nesterov accelerated gradient optimizer that build upon momentum
        model: the model we want to optimize the parameter on (this is a line right now)
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the learning rate for the step that weights update will take
        decay_factor: determines the relative contribution of the current gradient and earlier gradients to the weight change
        max_num_iteration: the number of iteration before we stop updating
    """
    
    # These are needed to keep track of the previous gradient
    g = [0 for _ in range(len(model.weights))] 
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)

        # Calculate the gradient for w0 by predicting where the ball will be (approximatively)
        for idx, gradient in enumerate(model.derivate(x,y)):
            
            # Here we need to do a bit of gymnastic because of how the code is setup
            # We need to save the parameters state, modify it, do the simulation and then reset the parameter state
            # The update happen in the next section
            prev_weight = model.weights[idx]
            model.weights[idx] = decay_factor*gradient
            g[idx] = decay_factor*g[idx] + learning_rate*gradient
            model.weights[idx] = prev_weight
            
            # Update the model parameter
            model.weights[idx] = model.weights[idx] - g[idx]
            
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)

In [ ]:
# Here we have a simple line with intercept = 0 and slope = 1
xs = [1,2,3,4,5,6,7]
ys = [1,2,3,4,5,6,7]

print("Target: intercept = 0 and slope = 1")


# # Gradient Descent
# model = Line()
# print("Gradient Descent: ")
# gd(model, xs, ys)
# print(model)

# Stochastic Gradient Descent
model = Line()
print("Stochastic Gradient Descent: ")
sgd(model, xs, ys)
print(model)

# # Stochastic Gradient Descent with Momentum
# model = Line()
# print("SGD + Momentum: ")
# sgd_momentum(model, xs, ys)
# print(model)


# # Adagrad
# model = Line()
# print("Adagrad")
# adagrad(model, xs, ys)
# print(model)

# # RMSprop
# model = Line()
# print("RMSprop")
# rmsprop(model, xs, ys)
# print(model)

# # Adam
# model = Line()
# print("Adam")
# adam(model, xs, ys)
# print(model)

# # Nesterov Accelerated Gradient
# model = Line()
# print("Nesterov Accelerated Gradient")
# nesterov(model, xs, ys)
# print(model)


Target: intercept = 0 and slope = 1
Stochastic Gradient Descent: 
Iteration 0
y = [0.85067745] + [0.45612767]*x
Iteration 100
y = [0.61852357] + [0.86383968]*x
Iteration 200
y = [0.41457415] + [0.90720916]*x
Iteration 300
y = [0.3058466] + [0.94680929]*x
Iteration 400
y = [0.21054131] + [0.94893116]*x
Iteration 500
y = [0.14755623] + [0.95979954]*x
Iteration 600
y = [0.10469542] + [0.97817653]*x
Iteration 700
y = [0.07060464] + [0.98570143]*x
Iteration 800
y = [0.04491438] + [0.99181598]*x
Iteration 900
y = [0.03322988] + [0.99324186]*x
y = [0.02414387] + [0.99522709]*x


In [ ]:
[np.random.uniform(0,1,1) for _ in range(2)]

[array([0.09642559]), array([0.12228897])]

In [ ]:
perm = permutation(5)
perm

array([1, 3, 4, 0, 2])

In [ ]:
x = [1, 2, 3, 4]
y = [5, 6, 7, 8]

for i in zip(x,y):
  print(i)

(1, 5)
(2, 6)
(3, 7)
(4, 8)


# Stohastic GD

In [ ]:
xs = [1,2,3,4,5,6,7]
ys = [1,2,3,4,5,6,7]

In [ ]:
# fx = weights[0] + weights[1]*x

# y = w0 + w1*x

def fx(weights, x):
  return weights[0] + weights[1]*x

In [ ]:
def stochastic_sample(xs, ys):
  
  id = np.random.randint(len(xs))
  x = xs[id]
  y = ys[id]

  return x, y

In [ ]:
# loss = (y-y_hat)**2
# dl/dw0 = 2(y-y_hat)
# dl/dw1 = 2x(y-y_hat)

def dx_w0(x, y, y_hat):
  return 2*(y_hat-y)

def dx_w1(x, y, y_hat):
  return 2*x*(y_hat-y)

In [ ]:
# Update at every data point
def sgd(xs, ys, weights, derivative_funcs, max_epochs = 1000, lr = 0.001):
  
  for i in range(max_epochs):
    
    x, y = stochastic_sample(xs, ys)

    y_hat = fx(weights, x)

    weights[0] = weights[0] - lr * derivative_funcs[0](x, y, y_hat)
    weights[1] = weights[1] - lr * derivative_funcs[1](x, y, y_hat)

    if i % 100 == 0:
      print(f"Equation ---> y = {weights[0]} + {weights[1]}*x")

In [ ]:
  #  total = 0
  #   for x,y in zip(xs,ys):
  #       yhat = evaluate(x)
  #       total = total + dx(x, y, yhat)
    
  #   gradient = total/N

In [ ]:
weights = [np.random.rand(), np.random.rand()]
derivative_funcs = [dx_w0, dx_w1]

In [ ]:
sgd(xs, ys, weights, derivative_funcs)

Equation ---> y = [0.11524274] + [0.61397645]*x
Equation ---> y = [0.17901088] + [0.95703724]*x
Equation ---> y = [0.1731139] + [0.96584744]*x
Equation ---> y = [0.1662497] + [0.96440353]*x
Equation ---> y = [0.15917359] + [0.96762779]*x
Equation ---> y = [0.15220276] + [0.96751028]*x
Equation ---> y = [0.14518989] + [0.97015811]*x
Equation ---> y = [0.14017751] + [0.97052684]*x
Equation ---> y = [0.13538963] + [0.97549695]*x
Equation ---> y = [0.12994668] + [0.97420925]*x


# NAG SGD

In [ ]:
weights = [np.random.rand(), np.random.rand()]
derivative_funcs = [dx_w0, dx_w1]

In [ ]:
# update rule uses uw and wb, that store history of all gradients, 
# multiplied by a decay constant + learning_rate * new gradient

def nagsgd(xs, ys, weights, derivative_funcs, max_epochs = 100, lr = 0.00000001, decay = 0.99):
  
  uw0, uw1 = 0, 0

  for i in range(max_epochs):
    
    x, y = stochastic_sample(xs, ys)

    y_hat = fx([weights[0]-uw0, weights[1]-uw1], x)

    uw0 += decay*uw0 + derivative_funcs[0](x, y, y_hat)
    uw1 += decay*uw1 + derivative_funcs[1](x, y, y_hat)

    weights[0] = weights[0] - lr*uw0
    weights[1] = weights[1] - lr*uw1

    if i % 10 == 0:
      print(f"Equation ---> y = {weights[0]} + {weights[1]}*x")

In [ ]:
msgd(xs, ys, weights, derivative_funcs)

Equation ---> y = 0.8856692219926035 + 0.19674179881252443*x
Equation ---> y = 168587.0020745633 + 639823.7147549607*x
Equation ---> y = 5.256896916450054e+17 + 3.4670931793860675e+18*x
Equation ---> y = 6.351180658560999e+31 + 4.6249098522221065e+32*x
Equation ---> y = 1.8243817606364102e+46 + 1.265645851062569e+47*x
Equation ---> y = -6.309875370762533e+57 + 2.603935315057803e+58*x
Equation ---> y = 1.6515712805958813e+72 + 6.480863548897396e+72*x
Equation ---> y = 9.19209667345592e+87 + 2.4906030703807563e+88*x
Equation ---> y = 3.691887757674088e+100 + 5.355338162516061e+100*x
Equation ---> y = 1.7683408182771098e+113 + 1.0849834907565852e+114*x


# ADAM